In [9]:
import numpy as np
import scipy.linalg as la
import time
import bezout_5 as bz

#TEX_DIR = '/home/jp/Documents/Bezout/bezout/tex/txt'
TEX_DIR = '../tex/txt'

deg = [2, 2, 2]

with open(TEX_DIR+'/deg.txt', 'w') as f:
    f.write(str(deg))
    
t = 2
m = 16000
n = len(deg)

Field = QQ
R = PolynomialRing(Field, 'x', n)
x = R.gens()
xx = [x[0]**0] + list(x)

fshape = [d+1 for d in deg]
dx = [(i+1)*deg[i] for i in range(n)]
dy = [(n-i)*deg[i] for i in range(n)]
fn, Dx, Dy = factorial(n), prod(dx), prod(dy)
with open(TEX_DIR+'/Dx.txt', 'w') as f:
    f.write("{0:d}".format(Dx))


P = [bz.rand_poly(n-1, m, deg, t, x) for i in range(n)] + xx
#P = load('P.sobj')

save(P, 'P')
bz.P2txt(n, deg, P, TEX_DIR)
F = [bz.poly2prism(fshape, p) for p in P]

TEX_DIR

t = time.clock()
Gx, Gy, Hx, Hy = bz._GH(n, fn, deg, dx, dy)
H, K = bz._HK(n, Hx, Hy)
J = bz._J(Dx, Dy, F, n, fshape, dx, dy, Gx, Gy)
C = bz._C(n, Dx, Dy, J)
B = bz._B(n, Dx, Dy, H, K, C)

construction_B_time = time.clock() - t
with open(TEX_DIR+'/construction_B_time.txt', 'w') as f:
    f.write("{0:.4f}".format(construction_B_time))

#B = bz.block_triang(n, Dx, Dy, dx, dy, deg, B)
BB = []
for k in range(n+1):
        Bk = matrix(Field, B[k])
        BB.append(Bk[:, :])
bz.bz2txt(n, TEX_DIR, BB)

"""
computing rank of B0
"""
t = time.clock()
r0 = rank(BB[0])
rank_B0_time = time.clock() - t

with open(TEX_DIR+'/rank_B0.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/rank_B0_time.txt', 'w') as f:
    f.write("{0:.4f}".format(rank_B0_time))
print("sage_rank = {0:d}".format(r0))

bb = []
for k in range(n+1):
    bb.append(np.array(BB[k], dtype=float))

b0 = bb[0]
numpy_rank = np.linalg.matrix_rank(b0)
print("numpy_rank = {0:d}".format(numpy_rank))

"""
reduction of Bezoutian matrices
"""
epsi = abs(b0).max()/1e5

t = time.clock()
nb_relations = 1
while nb_relations > 0:
    bb, r0, nb_relations = bz.iteration(bb, r0, epsi)
    print bb[0].shape, r0, nb_relations

sage_rank = 42
numpy_rank = 42
(48, 48) 42 0


In [10]:
Dxn = np.zeros((Dx, n), dtype=int)
for j in range(n):
    K = 1
    for k in range(n):
        if k == j:
            K = np.kron(np.array(range(dx[k])), K)
        else:
            K = np.kron(np.ones(dx[k]), K)
    Dxn[:, j] = K

In [11]:
def BB2p(i, idy, n, Dx, BB, xx):
    s = 0.0*xx[0]
    for k in range(Dx):
        m = bb[i][k, idy]
        for j in range(n):
            m *= xx[j+1]^Dxn[k, j]
        s += m
    return s

In [20]:
deltas = []
for idy in range(bb[0].shape[1]):
    p0 = BB2p(0, idy, n, Dx, BB, xx)
    for j in range(1, n+1):
        delta = BB2p(j, idy, n, Dx, BB, xx) - xx[j]*p0
        if delta != 0:
            #print np.array(delta.reduce(GB).coefficients())
            print delta.reduce(GB)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [115]:
print "bbt"
bbt = []
for k in range(n+1):
    bbt.append(bb[k].T)
nb_relations = 1
while nb_relations > 0:
    bbt, r0, nb_relations = bz.iteration(bbt, r0, epsi)
    print bbt[0].shape, r0, nb_relations
reductions_time = time.clock() - t
with open(TEX_DIR+'/bezout_dim.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/reductions_time.txt', 'w') as f:
    f.write("{0:.4f}".format(reductions_time))

"""
Numerical compation of the roots
"""
t = time.clock()
bbt_roots = bz.bbt2roots(bbt, epsi, r0)
compute_roots_time = time.clock() - t
with open(TEX_DIR+'/compute_roots_time.txt', 'w') as f:
    f.write("{0:.4f}".format(compute_roots_time))

test_bbtr = bz.roots_test(P, x, bbt_roots)
test_roots = np.sort(test_bbtr[:])

np.savetxt(TEX_DIR+'/test_roots.txt', test_roots,  fmt='%1.3e')

hist, bin_edges = np.histogram(np.log10(test_roots), bins='scott')

sage_rank = 9
numpy_rank = 9
(18, 17) 9 1
(18, 17) 9 0
bbt
(17, 18) 9 0


In [66]:
with open(TEX_DIR+'/histogram.txt', 'w') as f:
    for k in range(len(hist)):
        nb_roots = hist[k]
        left_bin, right_bin = bin_edges[k], bin_edges[k+1]
        if nb_roots > 0:
            if k < len(hist)-1:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\\\\\n".format(left_bin, right_bin, nb_roots))
            else:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\n".format(left_bin, right_bin, nb_roots))

sage_rank = 376
numpy_rank = 374
(384, 384) 376 0
bbt
(384, 384) 376 0


In [12]:
t = time.clock()
GB, grobner_dim = bz.compute_grobner(R, P, n)
grobner_time = time.clock() - t
bz.gb2txt(TEX_DIR, GB)

with open(TEX_DIR+'/grobner_dim.txt', 'w') as f:
    f.write("{0}".format(grobner_dim))

with open(TEX_DIR+'/grobner_time.txt', 'w') as f:
    f.write("{0:.4f}".format(grobner_time))

In [13]:
grobner_dim

42

In [ ]:
q, r, p = la.qr(bbt[0], pivoting=True)
sbbt = [bbt[k][:, p[:r0]] for k in range(n+1)]
xxx = [np.linalg.lstsq(sbbt[0], sbbt[k])[0] for k in range(1,n+1)]

In [83]:
[la.norm(xxx[i].dot(xxx[j]) - xxx[j].dot(xxx[i]), 'fro')/la.norm(xxx[i], 'fro')/la.norm(xxx[j], 'fro') \
    for i in range(n) for j in range(n) if i < j]

[2.9813787674884366e-06, 2.8259649958306715e-07, 1.8678442398423832e-07]

In [82]:
[la.norm(xxx[i].dot(xxx[j]) - xxx[j].dot(xxx[i]), 'fro')/la.norm(xxx[i].dot(xxx[j]), 'fro') \
    for i in range(n) for j in range(n) if i < j]

[0.022979743176457408, 0.00047141440582469755, 0.00012569316737470675]

In [90]:
v = np.random.randn(xxx[0].shape[0])
v /= la.norm(v)
la.norm(xxx[0].dot(v))

52650.43522620559

In [68]:
def numpy_X2m(xxx, v, m):
    n = len(xxx)
    r = v
    degs = m.degrees()
    for j in range(n):
        xj = xxx[j]
        for k in range(degs[j]):
            r = r.dot(xj)
    return r

In [69]:
def numpy_X2p(xxx, p):
    dim = xxx[0].shape[0]
    pm = p.monomials()
    pc = p.coefficients()
    v = np.random.randn(dim)
    v /= la.norm(v)
    s = np.zeros(dim)
    for k in range(len(pc)):
        m = pm[k]
        s += pc[k]*numpy_X2m(xxx, v, m)
    return s

In [7]:
with open(TEX_DIR+'/bezout_exact.txt', 'w') as f:
    X = matrix(ZZ, Dx, 0)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    Y, Y_time = bz.BB2Y(BB, X_ortho)
    f.write("nb de Y-relations = {0:d}\n".format(Y.nrows()))

    
    K = bz.Y2K(BB[0], X_ortho, Y)
    X_dim = Dx - K.nrows()
    f.write("X_dim = {0:d}\n".format(X_dim))

    
    Y_ortho = Y.right_kernel_matrix().LLL()
    X, X_time = bz.BB2X(BB, Y_ortho)
    N = bz.X2N(BB[0], Y_ortho, X)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    f.write("nb de X-relations = {0:d}\n".format(X.ncols()))

    
    XBBY = []
    for k in range(n+1):
        XBBY.append(X_ortho.transpose()*BB[k]*Y_ortho.transpose())
    bezout_exact_dim = rank(XBBY[0])
    f.write("bezout_exact_dim = {0:d}\n".format(bezout_exact_dim))

    
    K_ortho = XBBY[0].kernel().basis_matrix().right_kernel_matrix()
    N_ortho = N.kernel().basis_matrix().transpose()

    KBBN = []
    for k in range(n+1):
        KBBN.append(K_ortho*XBBY[k]*N_ortho)

    XX = []
    for k in range(n):
        xx = KBBN[0].solve_right(KBBN[k+1])
        XX.append(xx)

    test_XX = [norm(bz.X2p(XX, Field, p)) for p in P[:n]]
    
    f.write("test_XX = {0:s}".format(test_XX))


In [72]:
kbbn = [np.array(KBBN[0], dtype=float) for k in range(n+1)]
num_x_exact = [la.solve(KBBN[0], KBBN[k]) for k in range(1, n+1)]

In [121]:
sage: x,y,z = QQ['x,y,z'].gens()
sage: I = ideal(x^5 + y^4 + z^3 - 1,  x^3 + y^3 + z^2 - 1)
sage: B = I.groebner_basis()

In [128]:
sage: f,g,h = B
sage: (2.1*x^3).reduce(B)

-2.10000000000000*y^3 - 2.10000000000000*z^2 + 2.10000000000000